2013 年 SECCON CTF quals 中的 [Cryptanalysis](https://github.com/sonickun/ctf-crypto-writeups/blob/master/2013/seccon-ctf-quals/cryptanalysis/20140127213558.png)
## 题目描述
$$
y^2=x^3+1234577x+3213242 \pmod {7654319}
$$
$$
base:(5234568, 22877417)
$$
$$
PublicKey = SecretKey \times base
$$
$$
crypteddata = (rand() \times base, pbin + rand() \times PublicKey)
$$
$$
PublicKey:(2366653, 1424308)
$$
$$
crypteddata:[(5081741, 6744615), (610619, 6218)]
$$
$$
plain = (x, y)
$$
$$
x+y = ???????(=Password)
$$

这道题没有经过任何的变式，和教科书上的 ECC 形式基本完全一致。

a = 1234577

b = 3213242

M = 7654319

生成元 base = E([5234568, 2287747])

公钥 PublicKey = $SecretKey \times base $= E([2366653, 1424308])

密文 (C1, C2)
 = $(rand() \times base, pbin + rand() \times PublicKey)$

= [(5081741, 6744615), (610619, 6218)]

## 题目分析
### ECC 分析思路

<img src="image/ECC分析思路.png" width =550>
<p align="center">
ECC分析思路
</p>

<img src="image/ECC破解思路.png" width = 500>
<p align="center">
ECC破解思路
</p>

综上，用暴力破解的方法，根据公钥破解私钥

当时在学 ECC 的是时候，虽然每个步骤都懂了，也能进行加密和解密，但是没有解析这一道题目，自己其实没有真正领略 ECC 的加密核心。
所以算是很有收获~


### 代码构建
#### 已知条件收集
```python
a = 1234577
b = 3213242
M = 7654319
base = (5234568, 2287747)
pub = (2366653, 1424308)
crypt = [(5081741, 6744615), (610619, 6218)]
```

#### 加法运算
椭圆曲线下的加法运算没有现成的工具，需要自己实现。

同时，算法一定已经给出了需要的参数：椭圆曲线的参数 a, b, M，有限域的本原元 base。


In [17]:
a = 1234577
b = 3213242
M = 7654319
def add(A, B, a, b, M):
    if A == (0, 0): return B
    if B == (0, 0): return A
    x1, y1 = A
    x2, y2 = B
    if A != B:
        λ = (y2 - y1) * pow((x2 - x1), M-2, M)
    else:
        λ = (3*x1*x1 + a) * pow(2*y1, M-2, M)
    x3 = λ * λ - x1 - x2
    y3 = λ * (x1 - x3) - y1
    return (x3 % M, y3 % M)

由于 M 是素数，所以由费马小定理可知 $(x_2 - x_1)^{M-1} = 1 \pmod {M}$，

所以 $x_2 - x_1$ 的乘法逆元是 $(x_2 - x_1)^{M-2}$，同理，$y_2 - y_1$ 的乘法逆元是 $(y_2 - y_1)^{M-2}$.

#### 爆破
枚举密钥 x 使得 $x·Base == pub$

In [20]:
base = (5234568, 2287747)
pub = (2366653, 1424308)

X = (0, 0) # 初始值
for i in range(M):
    if X == pub:
        secret = i
        print("Secret:", str(i))
        break
    X = add(X, base, a, b, M)
    print(i)
# Secret: 1584718

Error: Jupyter cannot be started. Error attempting to locate jupyter: Select an Interpreter to start Jupyter

#### 由密钥进行解密

In [27]:
cipher = [(5081741, 6744615), (610619, 6218)]
xC1 = (0, 0)
c1, c2 = cipher[0], cipher[1]
for i in range(secret):
    xC1 = add(xC1, c1, a, b, M)

plaintext = add(c2, (xC1[0], -xC1[1]), a, b, M)
print(plaintext)
print("x+y=%d"%(plaintext[0]+plaintext[1]))
# (1629548, 7145227)

(2171002, 3549912)
x+y=5720914


### 封装暴力破解函数

In [50]:
%%writefile ECC-decrypt.py
# encoding=utf-8
def add(A, B, a, b, M):
    if A == (0, 0): return B
    if B == (0, 0): return A

    x1, y1 = A
    x2, y2 = B

    if A != B:
        λ = (y2 - y1) * pow((x2 - x1), M-2, M)
    else:
        λ = (x1*x1*3 + a) * pow(2*y1, M-2, M)
    
    x3 = λ * λ - x1 - x2
    y3 = λ * (x1 - x3) - y1
    
    return (x3 % M, y3 % M)

def blasting(base, pub, M):
    X = (0, 0) # 初始值
    for i in range(M):
        if X == pub:
            secret = i
            break
        X = add(X, base, a, b, M)

    return secret if X == pub else -1

def decrypt(a, b, M, base, pub, cipher):
    secret = blasting(base, pub, M)
    print(secret)
    xC1 = (0, 0)
    c1, c2 = cipher[0], cipher[1]
    for i in range(secret):
        xC1 = add(xC1, c1, a, b, M)

    plaintext = add(c2, (xC1[0], -xC1[1]), a, b, M)
    return plaintext
    # print("x+y=%d"%(plaintext[0]+plaintext[1]))

if __name__ == "__main__":
    a = 1234577
    b = 3213242
    M = 7654319
    base = (5234568, 2287747)
    pub = (2366653, 1424308)
    cipher = [(5081741, 6744615), (610619, 6218)]
    plaintext = decrypt(a, b, M, base, pub, cipher)
    print("x+y=%d"%(plaintext[0]+plaintext[1]))
    print()

Overwriting ECC-decrypt.py


In [51]:
%run ECC-decrypt.py

1584718
x+y=5720914



## 其他收获

### 小错误
#### 1.print 格式化
错误：`print("x_y=%d"%pliantext[0]+pliantext[1])`
正确点拨：这个是 C 语言有一点点区别，需要将格式化字符串后面的用 `%(..)` 隔离开来。
```python
print("x_y=%d"%pliantext[0]+pliantext[1])
```

#### 2.Jupyter VSCode
##### 2.1 LaTeX
Jupyter notebook 的 LaTeX 中貌似不支持用 `\\` 表示换行符，否则网页端打开对应的 `.ipynb` 的时候会有对话框提示错误
##### 2.2 图片
`![]()` 或者 `<img src="">`在 VSCode 的 `.ipynb` 的 Markdown 代码块中是不支持的。
但是在 Jupyter notebook 中是可以的。

### Tips
#### 1.IPython魔术命令
%timeit 、%time、 %reset、%run *.py

%timeit
多次执行一条语句，并返回平均时间，

%%timeit
多次执行多条语句，并返回平均时间，

%time
返回执行一条语句的时间，

%%time
返回执行多条语句的时间，

In [52]:
%%time
import time
for i in range(100):
    time.sleep(1)

Wall time: 1min 40s


In [54]:
%time
import time
for i in range(100):
    time.sleep(1)
# 这样是无效的


Wall time: 0 ns


In [ ]:
import time
%time time.sleep()

#### 2.python中`__name__`的使用
参考：https://www.cnblogs.com/1204guo/p/7966461.html

1. __name__是一个变量。前后加了爽下划线是因为是因为这是系统定义的名字。普通变量不要使用此方式命名变量。
2. Python有很多模块，而这些模块是可以独立运行的！这点不像C++和C的头文件。
3. import的时候是要执行所import的模块的。
4. __name__就是标识模块的名字的一个系统变量。这里分两种情况：假如当前模块是主模块（也就是调用其他模块的模块），那么此模块名字就是__main__，通过if判断这样就可以执行“__mian__:”后面的主函数内容；假如此模块是被import的，则此模块名字为文件名字（不加后面的.py），通过if判断这样就会跳过“__mian__:”后面的内容。


#### 3. 条件表达式的多种实现方法
1.常规
```python
if a>b:
    c = a
else:
    c = b
```
2.一行（类似于 `[C] ? : `三目运算符）
```python
c = a if a>b else b # 先执行中间的if，如果返回True，就是左边，False是右边。
```
我们在 `add()` 函数中用了这个。

3.二维列表
```python
c = [b,a][a>b] #实际是[b,a][False]，因为False被转换为0，所以是[1,2][0]，也就是[1]
    # False返回第一个，True 返回第一个。
```